In [1]:
#!/usr/bin/env python3
from collections import Counter

#from csv_combinator import Combinator
#from data_extraction import *
#from pepper_arcs.arcs import *
#from data_extraction.clean_csv import cleancsv, WrapperGetPart

%run 'Code csv combinator.ipynb'
%run 'Code data_extraction.ipynb'
%run 'Code arcs.ipynb'

import pandas as pd

REGEX = r'(?<=P)[0-9]*'
NEW_RECORDINGS_FOLDER = r'new_recording/'
NEW_RECORDINGS_COMBINED = r'new_data/combined.csv'
CLEANED_ROTATED_DATA = r'new_data/cleaned_rotated_combined.csv'
SIDED_DATA = r'new_data/sided_data.csv'
ARCS = r'new_data/combined_arcs.csv'
PNUM_COLUMN_NAME = 'pNum'
ENUM_COLUMN_NAME = 'eNum'

def remove_multiple_bodies(dataToFilter):
    per_pnum = [x for _, x in dataToFilter.groupby(dataToFilter.pNum)]
    df_list = []

    for pnum in per_pnum:
        per_enum = [x for _, x in pnum.groupby(pnum.eNum)]
        for enum in per_enum:
            trackings = [x for _, x in enum.groupby(enum.trackingId)]
            df_list.append(max(trackings, key=len))

    return pd.concat(df_list)
    
def run():
    c = Combinator(NEW_RECORDINGS_FOLDER, NEW_RECORDINGS_COMBINED)
    c.combine(REGEX, PNUM_COLUMN_NAME, ENUM_COLUMN_NAME)
    removed_bodies = remove_multiple_bodies(NEW_RECORDINGS_COMBINED)

    per_pnum1 = [x for _, x in removed_bodies.groupby(removed_bodies.pNum)]
    rotated = None

    for person in per_pnum1:
        print(person.iloc[0].pNum)
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        for exercise in per_enum:
            print(exercise.iloc[0].eNum)
            rotated_e = rotate_body(exercise)
            if type(rotated) == type(None):
                rotated = rotated_e
            else:
                rotated= rotated.append(rotated_e)
    
    try:
        old_rotated = pd.read_csv(CLEANED_ROTATED_DATA)        
        new_rotated = old_rotated.append(rotated)
    except:
        new_rotated = rotated
    new_rotated.to_csv(CLEANED_ROTATED_DATA)



    # Do Thijs' stuff
    per_pnum2 = [x for _, x in rotated.groupby(rotated.pNum)]
    combined_with_sides = None
    for person in per_pnum2:
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        if len(per_enum) == 3:
            for exercise in per_enum:
                try:
                    enum_ = exercise.iloc[0].eNum
                    exercise = exercise.reset_index(drop=True)
                    cut_exercise = cleancsv(exercise, enum_)
                    cut_exercise = cut_exercise.reset_index(drop=True)
                    cut_exercise = WrapperGetPart(cut_exercise, enum_)                    
                    if type(combined_with_sides) == type(None):
                        combined_with_sides = cut_exercise
                    else:
                        combined_with_sides = pd.append(cut_exercise)
                except:
                    print("EXCEPTION!")
                    continue

    try:
        old_sided = pd.read_csv(SIDED_DATA)
        new_sided = old_sided.append(combined_with_sides)
    except:
        new_sided = combined_with_sides        
    new_sided.to_csv(SIDED_DATA)

    # Calculate arcs
    per_pnum3 = [x for _, x in combined_with_sides.groupby(combined_with_sides.pNum)]
    all_arcs = None
    for person in per_pnum3:
        pnum = person['pNum'].iloc[0]
        print(pnum)
        per_enum = [x for _, x in person.groupby(person.eNum)]
        for exercise in per_enum:
            enum_ = exercise['eNum'].iloc[0]
            per_side = [x for _, x in exercise.groupby(exercise.Side)]
            for sided in per_side:
                Side = sided['Side'].iloc[0]
                arcs = get_arcs(sided, enum_, Side)
                if not Side == '':
                    arcs = get_arcs(sided, enum_, Side)
                    arcs['pNum'] = pnum
                    arcs['eNum'] = enum_
                    arcs['Side'] = Side
                    if type(all_arcs) == type(None):
                        all_arcs = arcs
                        print("all_arcs is None")
                    else:
                        all_arcs = all_arcs.append(arcs)
                        print("APPENDED")

    try:
        old_arcs = pd.read_csv(ARCS)
        new_arcs = old_arcs.append(all_arcs)
    except:
        new_arcs = all_arcs

    new_arcs.to_csv(r'/data/pepper/upto_92.csv')

    print("jwz")

    
def try_read_csv(fileName):
    try:
        df = pd.read_csv(fileName)
    except:
        df = pd.DataFrame()
    return df
        
def combine_data(input_folder, current_combined):
    if len(current_combined) > 0:
        currentPersons = current_combined.pNum.unique()
    else:
        currentPersons = []
        
    combined_data = current_combined
    if not os.path.exists(input_folder):
        raise FileNotFoundError()
    for root, dirs, files in os.walk(input_folder, topdown=True):
        dirs.sort()
        files.sort()
        fileCounter = 1        
        regResult = re.search(REGEX, root)
        if type(regResult) != type(None):
            personNumber = int(regResult.group(0))
            if not personNumber in currentPersons: 
                for file in files:
                    fullpath = root + '/' + file
                    if fullpath[-3:] == 'csv':           
                        new_data = pd.read_csv(fullpath)
                        new_data[PNUM_COLUMN_NAME] = personNumber;
                        new_data[ENUM_COLUMN_NAME] = fileCounter;
                        combined_data = combined_data.append(new_data)
                        fileCounter+=1   
    print("combining data finished")
    return combined_data
      
def clean_data(dataToClean):
    per_pnum = [x for _, x in dataToClean.groupby(dataToClean.pNum)]
    df_list = []

    for pnum in per_pnum:
        per_enum = [x for _, x in pnum.groupby(pnum.eNum)]
        for enum in per_enum:
            trackings = [x for _, x in enum.groupby(enum.trackingId)]
            df_list.append(max(trackings, key=len))

    single_bodies = pd.concat(df_list) 
    
    #TODO
    # Replace commas with points in all values
    #row = dict(map(lambda k: (k[0], str(k[1]).replace(',', '.')), row.items()))
    
    print("cleaning finished")
    return single_bodies
    
def cut_frames_add_moving_sides(dataToCut):
    # Do Thijs' stuff
    per_pnum2 = [x for _, x in dataToCut.groupby(dataToCut.pNum)]
    cut_with_sides = None
    for person in per_pnum2:
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        if len(per_enum) == 3:
            for exercise in per_enum:
                try:
                    enum_ = exercise.iloc[0].eNum
                    exercise = exercise.reset_index(drop=True)
                    cut_exercise = cleancsv(exercise, enum_)
                    cut_exercise = cut_exercise.reset_index(drop=True)
                    cut_exercise = WrapperGetPart(cut_exercise, enum_)                    
                    if type(combined_with_sides) == type(None):
                        cut_with_sides = cut_exercise
                    else:
                        cut_with_sides = pd.append(cut_exercise)
                except:
                    print("EXCEPTION!")
                    continue 
    print("cutting frames finished")
    return cut_with_sides    

def rotate_data(dataToRotate):
    per_pnum1 = [x for _, x in dataToRotate.groupby(dataToRotate.pNum)]
    rotated = None

    for person in per_pnum1:
        print(person.iloc[0].pNum)
        per_enum = [x for _, x in person.groupby(person['eNum'])]
        for exercise in per_enum:
            print(exercise.iloc[0].eNum)
            rotated_e = rotate_body(exercise)
            if type(rotated) == type(None):
                rotated = rotated_e
            else:
                rotated= rotated.append(rotated_e)   
    print("rotating data finished")
    return rotated
    
def calculate_angles(dataToProcess):
    # Calculate arcs
    per_pnum3 = [x for _, x in dataToProcess.groupby(dataToProcess.pNum)]
    all_arcs = None
    for person in per_pnum3:
        pnum = person['pNum'].iloc[0]
        print(pnum)
        per_enum = [x for _, x in person.groupby(person.eNum)]
        for exercise in per_enum:
            enum_ = exercise['eNum'].iloc[0]
            per_side = [x for _, x in exercise.groupby(exercise.Side)]
            for sided in per_side:
                Side = sided['Side'].iloc[0]
                arcs = get_arcs(sided, enum_, Side)
                if not Side == '':
                    arcs = get_arcs(sided, enum_, Side)
                    arcs['pNum'] = pnum
                    arcs['eNum'] = enum_
                    arcs['Side'] = Side
                    if type(all_arcs) == type(None):
                        all_arcs = arcs
                        print("all_arcs is None")
                    else:
                        all_arcs = all_arcs.append(arcs)
                        print("APPENDED")
    print("calculating angles finished")
    return all_arcs
    

